<a href="https://colab.research.google.com/github/ValdazoAmerico/Similitud-de-autores-NLP-/blob/main/ejercicio_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [93]:
from keras.preprocessing.text import Tokenizer
from keras import preprocessing
import json
import numpy as np
from tensorflow import keras
from keras.preprocessing.text import text_to_word_sequence

In [94]:
import random

In [ ]:
##Funciones

In [95]:
def get_data(filename):
  with open(filename, "r", encoding='utf-8') as f:
    data = f.read()[50:424431]
  data = data.lower()
  return (data)

In [96]:
def create_index(texts, filename):
  words = texts.split()
  tokenizer = Tokenizer(num_words=100000)
  tokenizer.fit_on_texts(words)
  sequences = tokenizer.texts_to_sequences(words)
  word_index = tokenizer.word_index
  print(f"Found {len(word_index)} unique words")
  with open (filename, "w") as f:
    json.dump(word_index,f, indent=4)

In [97]:
def get_index(filename):
  with open(filename, "r") as f:
    data = json.load(f)
  return (data)

In [98]:
def create_sents(text):
  sentences = text.split(".")
  return (sentences)

In [99]:
def padding_data(sentences,index,maxlen=25):
  new_sentences = []
  for sentence in sentences:
    sentence = text_to_word_sequence(sentence)
    new_sentence = []
    words = []
    for word in sentence:
      try:
        word = index[word]
      except:
        KeyError
        word = 0
      words.append(word)
    new_sentence.append(words)
    new_sentence = preprocessing.sequence.pad_sequences(new_sentence,maxlen=maxlen,padding='post')
    new_sentences.append(new_sentence[0])
  return (new_sentences)

In [100]:
def reverse_index(word_index):
  reverse_word_index = {value: key for (key, value) in word_index.items()}
  return (reverse_word_index)

In [101]:
def reconst_text(text, reverse_word_index):
  return(" ".join([reverse_word_index.get(i, "?") for i in text]))

In [102]:
def label_data(sentences, label):
  total_chunks = []
  for sentence in sentences:
    total_chunks.append((sentence, label))
  return (total_chunks)

In [103]:
def create_training(total_chunks, cutoff):
  random.shuffle(total_chunks)
  training_data = []
  training_labels = []
  testing_data = []
  testing_labels = []
  test_num = len(total_chunks)*cutoff
  x=0
  for entry in total_chunks:
    if x > test_num:
      testing_data.append(entry[0])
      testing_labels.append(entry[1])
    else:
      training_data.append(entry[0])
      training_labels.append(entry[1])
    x = x+1
  training_data = np.array(training_data)
  training_labels = np.array(training_labels)
  testing_data = np.array(testing_data)
  testing_labels = np.array(testing_labels)
  return (training_data, training_labels, testing_data, testing_labels)

In [ ]:
##Creando el data de entrenamiento

In [104]:
o_file = 'oscar.txt'
o_data = get_data(o_file)
print(o_data[0:30])

an gray, by oscar wilde

this 


In [105]:
b_file = 'joyce.txt'
b_data = get_data(b_file)
print(b_data[0:30])

 joyce

this ebook is for the 


In [106]:
all_texts = o_data+b_data
create_index(all_texts, "word_index.json")

Found 15948 unique words


In [107]:
word_index = get_index("word_index.json")

In [108]:
o_sents = create_sents(o_data)

In [109]:
b_sents=create_sents(b_data)

In [110]:
o_padded = padding_data(o_sents, word_index,maxlen=25)

In [111]:
b_padded = padding_data(b_sents, word_index,maxlen=25)

In [112]:
print(o_sents[25])
print(o_padded[25])
print(reconst_text(o_padded[25], reverse_word_index))

  the artist
can express everything
[   1  527  103 1705  248    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0]
the artist can express everything ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?


In [113]:
reverse_word_index = reverse_index(word_index)

In [114]:
o_labelled = label_data(o_padded, 0)

In [115]:
o_labelled[0]

(array([  48,   90,   38, 2928, 1549,   72, 1697,   13,   16,    1,  498,
           2, 1906, 1550,   22,   43, 2439,    3,   15,  374,   43, 4799,
        3617,    0,    0], dtype=int32), 0)

In [116]:
b_labelled = label_data(b_padded, 1)

In [117]:
b_labelled[0]

(array([4300,   72, 1697,   13,   16,    1,  498,    2, 1906, 1550,   22,
          43, 2439,    3,   15,  374,   43, 4799, 3617,    0,    0,    0,
           0,    0,    0], dtype=int32), 1)

In [118]:
all_data = o_labelled+b_labelled

In [119]:
tt_data = create_training(all_data, cutoff=.95)
print(tt_data[0])

[[ 270  104   54 ...    0    0    0]
 [   4  116 4865 ...    0    0    0]
 [  75  178  338 ...    0    0    0]
 ...
 [ 103   10  239 ...    0    0    0]
 [   0    0    0 ...    0    0    0]
 [ 308  287  496 ...    0    0    0]]


In [120]:
print(tt_data[3])

[0 0 1 0 0 0 1 1 0 0 0 1 1 0 1 0 1 1 1 0 1 1 1 1 1 1 1 1 0 0 0 0 1 1 1 1 1
 1 1 1 1 1 1 0 1 1 1 0 0 0 0 1 1 1 1 1 1 0 0 1 1 0 1 0 1 0 1 0 0 1 0 0 0 0
 1 1 0 1 1 1 0 1 1 0 0 0 0 1 0 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 1 0 1 0 0 0
 1 1 1 1 1 1 1 1 0 0 1 0 1 0 1 1 0 1 0 1 0 0 1 0 0 0 0 1 1 1 0 1 1 1 1 1 0
 1 1 1 1 1 1 0 1 0 1 1 1 1 1 0 1 0 0 1 0 0 0 1 0 0 0 1 1 1 1 1 1 1 0 0 1 1
 0 1 1 1 1 1 0 1 1 1 1 1 1 0 0 0 0 0 1 1 0 0 0 1 0 1 1 0 0 1 1 1 1 1 1 1 0
 1 1 0 1 1 1 0 0 0 1 1 0 1 1 0 1 0 1 0 0 0 0 1 1 0 0 1 1 1 0 1 0 1 1 1 1 0
 1 0 1 1 1 0 0 0 1 0 1 1 0 0 1 1 0 1 0 1 1 0 1 1 1 1 1 0 0 0 1 0 1 0 1 0 1
 0 0 0 1 0 0 1 0 1 1 0 1 1 1 1 1 0 1 1 0 1 1 1 0 1 0 0 1 0 1 1 1 1 1 1 1 0
 1 1 0 0 1 1 1 0 1 1 0 0 0 0 1 1 1 1 1 0 1 1 1 0 1 1 1 1 0 1 0 1 1 0 0 0 0
 0 0 1 1 1 0 0 1 1 0 0 1 1 0 1 0 1 1 1 1 1 1 0 0 1 1 0 1 0 1 0 1 1 1 0 1 1
 0 1 1 0 0 1 1 0 0 1 1 1 1 0 1 1 1 1 0 1 0 0 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1
 1 1 1 0 0 0 1 1 1 1 1 1 1 0 1 1 0 0 0 0 1 1 1 0 1 0 0 0 0 1 0 0 0 0 1 1 0
 0 0 0 0 0 1 0 0 1 1 0 0 

In [ ]:
##Creamos el modelo

In [121]:
def create_model():
  model = keras.Sequential()
  model.add(keras.layers.Embedding(20000, 25))
  model.add(keras.layers.GlobalAveragePooling1D())
  model.add(keras.layers.Dense(16, activation="relu"))
  model.add(keras.layers.Dense(16, activation="tanh"))
  model.add(keras.layers.Dense(1, activation="sigmoid"))
  model.summary()
  model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])
  return (model)

In [ ]:
##Entrenamos el modelo

In [122]:
def train_model(model, tt_data, val_size=.3, epochs=1, batch_size=16):
  vals = int(len(tt_data[0])*val_size)
  training_data = tt_data[0]
  training_labels = tt_data[1]
  testing_data = tt_data[2]
  testing_labels = tt_data[3]
  
  x_val = training_data[:vals]
  x_train = training_data[vals:]

  y_val = training_labels[:vals]
  y_train = training_labels[vals:]

  fitModel = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val), verbose=1, shuffle=True)
  model_results = model.evaluate(testing_data,testing_labels)
  return (model)


In [123]:
model = create_model()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, None, 25)          500000    
_________________________________________________________________
global_average_pooling1d_4 ( (None, 25)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 16)                416       
_________________________________________________________________
dense_13 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 17        
Total params: 500,705
Trainable params: 500,705
Non-trainable params: 0
_________________________________________________________________


In [124]:
model  = train_model(model, tt_data=tt_data, epochs=5, batch_size=16)

Epoch 1/5
612/612 [==============================] - 6s 8ms/step - loss: 0.5715 - accuracy: 0.6941 - val_loss: 0.3366 - val_accuracy: 0.8568
Epoch 2/5
612/612 [==============================] - 5s 8ms/step - loss: 0.2528 - accuracy: 0.9000 - val_loss: 0.3040 - val_accuracy: 0.8754
Epoch 3/5
612/612 [==============================] - 5s 8ms/step - loss: 0.1559 - accuracy: 0.9413 - val_loss: 0.3332 - val_accuracy: 0.8592
Epoch 4/5
612/612 [==============================] - 5s 8ms/step - loss: 0.1235 - accuracy: 0.9570 - val_loss: 0.3392 - val_accuracy: 0.8626
Epoch 5/5
23/23 [==============================] - 0s 1ms/step - loss: 0.3660 - accuracy: 0.8680


In [130]:
model.save(path + 'models/ob_model')
model = keras.models.load_model(path + 'models/ob_model')

INFO:tensorflow:Assets written to: gdrive/MyDrive/models/ob_model/assets


In [127]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [128]:
path = "gdrive/MyDrive/"

In [131]:
##Testeamos el modelo

In [141]:
def test_model(text_chunks, reverse_word_index, model, cutoff=0):
  results=[]
  print('Analysis')
  for test in text_chunks:
    if len(test) > 2:
      print(test)
      predict = model.predict([test])
      if predict[0] > cutoff:
        print("Prediction: " + str(predict[0]))
        results.append((str(predict[0]), reconst_text(test, reverse_word_index)))
  return (results, cutoff)

In [137]:
def write_test(results, filename, name):
  with open(filename+'.txt', "w", encoding='utf-8') as f:
    f.write("*****TEST ON****")
    f.write(f"**********{name}""""""""")
    for result in results:
      f.write(str(result)+"\n")

In [138]:
t_file = "oscar2.txt"
t_text = get_data(t_file)
t_sents= create_sents(t_text)
t_padded = padding_data(t_sents, word_index, maxlen=25)

In [139]:
#Corremos el test

In [142]:
test_results = test_model(t_padded, reverse_word_index=reverse_word_index, model=model)

Analysis
[   38  2928  1549  7595    38 11961     0    72  1697    13    16     1
   498     2  1906  1550    22    43  2439     3    15   374    43  4799
  3617]
Prediction: [0.99999285]
[1551   11  221   11  118   46 4800  498   11  200    1 1219    2    1
 2440 2128 4801 4802   15   72 1697   46 4803   22 4804]
Prediction: [0.00028485]
[2128    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0]
Prediction: [0.9999115]
[    0     3     1  2440  2128  4803     0     0 12308   871  2440  2128
   455    66    48  7211  4810     2    72  1908    98     0     1  1764
     0]
Prediction: [0.9622183]
[  70    0 1897    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0]
Prediction: [0.99952143]
[   0   46    0 1897    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0]
Prediction: [0.9622183]
[   0    0 4804    0    

In [145]:
write_test(test_results[0], filename='sample_data', name="testing.txt")